<a href="https://colab.research.google.com/github/Bigote93/Chatbot_Alura_Sprint1/blob/main/Chatbot_Inteligente_Alura_Sprint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint 01 - Chatbot - Alura Latam

### (*) Configuracion ambiente

In [72]:
!pip install scipy
!pip install transformers

!pip install jellyfish


In [73]:
#Recursos y librerias
import pandas as pd
import re, os, random, pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
import jellyfish

from transformers import BertForSequenceClassification
from transformers import BertTokenizer

import torch



In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## (1) Lista de carga de verbos

In [75]:
#Instanciamos la lista de verbos
verbos_irregulares = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    verbos_irregulares = pickle.load(fichero_listaverbos)

In [76]:
#Instanciamos la lista de verbos
lista_verbos = []
#Direccion de lista de verbos
path_lista_verbos = '/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle'

#Comprobar el tamaño del archivo
tamaño_archivo = os.path.getsize(path_lista_verbos)

if tamaño_archivo == 0:
    print("El archivo está vacío.")
else:
    #Apertura de documentos
    fichero_listaverbos = open(path_lista_verbos, 'rb')

    #Cargar fichero
    lista_verbos = pickle.load(fichero_listaverbos)

## (2) Tratamiento de datos

### Raiz de los verbos

Función raiz que reciba una palabra y la compare con todas las palabras de la lista_verbos utilizando jaro_winkler , y que devuelva la palabra de lista_verbos con mayor similaridad a la palabra ingresada.

> Observación: Si la palabra encontrada, con mayor similaridad, no supera el radio de 0.93 entonces deberá regresar la palabra original.

In [77]:
def raiz_verbo(verbo_analizar):

  #Lista de verbos cercanos
  verbos_cercanos = []

  #Recorremos la lista de verbos
  for verbo in lista_verbos:
    #Condicionador de verbos y adjuntar a la lista
    if jellyfish.jaro_winkler_similarity(verbo_analizar,verbo) >= 0.93:
      #Verificamos si esta el verbo en la lista
      if verbo not in verbos_cercanos:
        verbos_cercanos.append(verbo)
    else:
      #Verificamos si esta el verbo en la lista
      if verbo_analizar not in verbos_cercanos:
        verbos_cercanos.append(verbo_analizar)

  return verbos_cercanos

#Aplicacion
raiz_verbo('Comir')

['Comir']

### Tratamiento de texto

Función tratamiento_texto que reciba una frase de texto y devuelva la misma frase pero sin acentuaciones, todo en minúscula y sin espacios en blanco adicionales.

Ejemplo:

```
# Ejemplo visualizacion

tratamiento_texto('¡Buen día!, ¿Cómo está todo hoy?')

>> '¡buen dia!, ¿como esta todo hoy?'

```



In [114]:
def tratamiento_texto(texto):

  # Convertimos el texto a minúsculas.
  if texto is not None:
    texto_minusculas = texto.lower()
    # Eliminamos los caracteres con tildes.
    texto_tratado = texto_minusculas.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    return texto_tratado
  else:
    return f'{texto}'


tratamiento_texto('Deseo comerme el mundo')

'deseo comerme el mundo'

### Terminacion de palabras

Función reemplazar_terminacion que reciba una palabra e identifique si la misma termina en alguna de las siguientes palabras: “es”, “me”, “as”, “te”, “ste”, si coincide, entonces que substituya esa terminación por la letra “r”.

In [79]:
def reemplazar_terminacion(palabra):

  terminaciones = ['ste','es','me','as','te',]
  #Verificar si tiene terminacion
  for terminacion in terminaciones:
    if terminacion in palabra[-len(terminacion):]:

      #Cambiar la terminacion de la palabra
      verbo_infinitivo = palabra[:-len(terminacion)]+'r'

      return verbo_infinitivo

reemplazar_terminacion('Comiste')

'Comir'

### Normalizacion de textos

Crear una funcion que identifique la categoria de la palabra.

>Observacion: Antes de realizarlo, debemos aplicar la funcion de puntuacion y tildes para limpiar la oracion

In [139]:
import spacy

#Función para devolver los tokens normalizados del texto
def normalizar(texto):

  # Almacenamos los tokens en una lista
  tokens=[]

  nlp = spacy.load('en_core_web_sm')

  # Instanciamos de acuerdo a las credenciales de tokenizar
  doc = nlp(texto)

  # Barrimos cada token del objeto tokenizado
  for t in doc:

    '''
    # Tomamos el primer parametro para buscar entre las claves de los verbos irregulares
       - Si existe la "key" (t.text) imprimira el valor asociado a la key
       - Si no existe la "key" (t.text) imprimira el valor indicado como segund valor (t.lemma_.split()[0]) que es la palabra directamente del texto
    '''
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])

    # Se eliminara los caracteres con el regex indicado
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)

    '''
    # Si el tipo de la palabra es reconocida dentro de "t.pos_" o asignada anteriormente en "lemma" y ademas esta presente en la lista verbos se realiza:
      - Verifica si es un verbo "VERB" aplicamos la terminacion de cambio de verbos "reemplazar_terminacion"
      - Luego aplicamos incorporacion al token mediante la busqueda de raiz del verbo en el diccionario "lista_verbos" con el metodo "raiz_verbo"
      - Antes de agregar, usamos la funcion "tratamiento_texto" para quitar puntuaciones y tildes

      - Si no es un verbo identificado como tipo "VERB" entonces solo quitamos tildes y puntuaciones y lo agregamos a tokens
    '''
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz_verbo(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))


  '''
  Fuera del recorrido de los token de las palabras, tomamos la lista de tokens incorporados y hacemos lo siguiente:
   - Se transforma la lista "tokens" a un diccionario donde cada elemento es una clave y su valor es None:
      lista_n = ['hola', 'mi','nombre','es','diego', 'si', 'es', 'italiano']
      dict.fromkeys(lista_n) -> {'hola': None, 'mi': None, 'nombre': None, 'es': None, 'diego': None}
   - Luego volvemos a transformar en lista eliminando los repetidos
      list(dict.fromkeys(lista_n))
   - Luego filtramos los valores None de la lista y lo volvemos a crear como lista
      list(filter(None, lista_n))
  '''

  tokens = list(filter(None, list(dict.fromkeys(tokens))))

  return tokens


normalizar('soy diego nalli y deseo ser el mejor campeon del mundo')

['ser',
 'diego',
 'nalli',
 'y',
 'deseo',
 'el',
 'mejor',
 'campeon',
 'del',
 'mundo']

## (3) Cargar base de documentos

### Cargar preguntas

In [81]:
# Generamos una ruta relativa para busqueda de elementos
direccion_relativa = '/content/drive/MyDrive/Chatbot'
dir_relativa_entrenamiento = direccion_relativa + '/dialogos'

# Barrido de documentos internos el folder
titulo_doc_list = [documento for documento in os.listdir(dir_relativa_entrenamiento) if documento.endswith('.txt')]

# Creamos listas para almacenar contenidos
lista_dialogos, lista_dialogos_respuesta, lista_dialogos_tipo = [], [], []

#Recorremos los documentos para almacenar las oraciones
for name_doc in range(len(titulo_doc_list)):

  # Apertura de documentos con los titulos almacenados
  doc = open(
      dir_relativa_entrenamiento + '/' + titulo_doc_list[name_doc],
      'r',
      encoding = 'utf-8',
      errors = 'ignore'
      )

  # Separar en lineas por cada texto
  lineas = doc.read().split('\n')

  # Carga de lineas del documento abierto
  for index, linea in enumerate(lineas):



    if index % 2 != 0: # Par: Respuesta necesaria
      lista_dialogos_respuesta.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              )
          )

      # Almacenar el tipo de linea
      lista_dialogos_tipo.append(titulo_doc_list[name_doc].replace('.txt', ''))

    else: # Impar: Pregunta a bot
      lista_dialogos.append(
          tratamiento_texto(
              re.sub(
                  r"[^\w\s+\-*/]",
                  '',
                  linea
                  )
              ))
print(f'El tamaño de dialogos es: {len(lista_dialogos)}')
print(f'El tamaño de respuestas es: {len(lista_dialogos_respuesta)}')
print(f'El tamaño de tipo es: {len(lista_dialogos_tipo)}')

El tamaño de dialogos es: 1152
El tamaño de respuestas es: 1152
El tamaño de tipo es: 1152


In [140]:
# Creamos dataframe "df_dialogo"
datos_dialogo = {
    'dialogo': lista_dialogos,
    'respuesta': lista_dialogos_respuesta,
    'tipo': lista_dialogos_tipo,
    'interseccion': 0,
    'jaro_winkler': 0,
    'probabilidad': 0
}

#Instanciamos el dataframe
df_dialogo = pd.DataFrame(datos_dialogo)

#Eliminamos los registros que se han repetido
df_dialogo = df_dialogo.drop_duplicates(keep = 'first')

#Eliminamos el index del dataframe
df_dialogo.reset_index(drop=True, inplace=True)

df_dialogo.sample(5)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
887,que es siuba python,siuba python es una biblioteca de analisis de ...,Otros,0,0,0
446,hola bot,hola como puedo ayudarte,Saludos,0,0,0
562,puedes explicar el concepto de odio,el odio es un sentimiento de aversion o repuls...,Sentimiento,0,0,0
294,podrias decirme tu nombre,claro mi nombre es madre y tu como te llamas,Nombre,0,0,0
622,guardas mi historial de conversaciones o mensa...,no como chatbot no almaceno ningun historial d...,Usuario,0,0,0


## (4) Buscar respuesta del Chatbot

### Comparacion de Textos

In [83]:
# Funcion para verificar si el usuario inicio un dialogo
def dialogo(pregunta_usuario, df):

  df_registros = df.copy()

  # Al igual que la normalizacion de preguntas, eliminamos caracteres como normalizados
  pregunta_user_tratada = re.sub(
      r"[^\w\s]",
      '',
      tratamiento_texto(pregunta_usuario)
      )

  # Crear un vectorizador y entrenarlo con tus datos
  vectorizador = TfidfVectorizer()
  tfidf_matrix = vectorizador.fit_transform(df_dialogo['dialogo'])

  # Funcion interseccion: Cantidad de palabras en comun
  def interseccion(texto_user, texto_comparar):

    #Transformamos a Set un string
    user_text = set(texto_user.split())
    df_text = set(texto_comparar.split())

    # Calculo de interseccion
    per_words_commun = len(user_text.intersection(df_text)) / len(user_text)

    return per_words_commun

  # Funcion similarity valor de similitud matematica
  def similarity(texto_user, texto_comparar):

    # Transformamos y obtenemos la union entre los registros
    text_user_vector = vectorizador.transform([texto_user])
    text_comparar_vector = vectorizador.transform([texto_comparar])

     # Calcular la similitud coseno entre el texto del usuario y el texto de comparación
    similarity_score = cosine_similarity(text_user_vector, text_comparar_vector)

    return similarity_score[0][0]


  # Recorremos cada fila para buscar cercania de pregunta con dialogo
  for idx, row in df_registros.iterrows():

    df_registros.at[idx, 'interseccion'] = interseccion(pregunta_user_tratada, row['dialogo'])
    df_registros.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(pregunta_user_tratada,row['dialogo'])
    df_registros.at[idx, 'similarity'] = similarity(pregunta_user_tratada, row['dialogo'])
    df_registros.at[idx, 'probabilidad'] = max(df_registros.at[idx, 'interseccion'], df_registros.at[idx, 'similarity'], df_registros.at[idx, 'jaro_winkler'])

  return df_registros


In [84]:
# Ejemplo:

df_registros = dialogo('Me tengo que retirar a comer, gracias por tu ayuda', df_dialogo)

df_registros.sort_values(by = 'probabilidad', ascending=False)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,similarity
1065,me alegro de haberte conocido hasta pronto,igualmente hasta pronto,Despedida,0.1,0.803102,0.803102,0.083425
958,me interesa curso de programacion,debes revisar el canal de elprofealejo en youtube,Otros,0.1,0.802756,0.802756,0.099812
1061,me alegra haberte podido ayudar hasta luego,gracias por todo hasta luego,Despedida,0.1,0.796589,0.796589,0.076288
1097,gracias por tu ayuda,no hay problema siempre estoy aqui para ayudarte,Agradecimiento,0.4,0.552041,0.756142,0.756142
1071,bueno me tengo que ir ya hasta la proxima,hasta la proxima,Despedida,0.3,0.741696,0.741696,0.329599
...,...,...,...,...,...,...,...
646,2 mas 2,no estoy autorizado a darte estos resultados,Otros,0.0,0.384354,0.384354,0.000000
952,xd lmao,jajaja,Otros,0.0,0.384354,0.384354,0.000000
492,saludos desde,hola espero que esten disfrutando de un dia ma...,Saludos,0.0,0.361329,0.361329,0.000000
744,dividir 5,no estoy autorizado a darte este tipo de infor...,Otros,0.0,0.353741,0.353741,0.000000


### Machine Learning

Clasificaremos ahora el modelo mediante un modelo de Transformers

In [ ]:
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#### Normalizando dialogos

In [141]:
# Normalizando frases
label_encoder = LabelEncoder()

# Normalizar cada dialogo
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(list(normalizar(x))))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']

df_dialogo

TypeError: ignored

In [136]:
lista_n = []
for elemento in df_dialogo['dialogo']:
  lista_n.append(normalizar(elemento))

lista_n

TypeError: ignored